In [1]:
import numpy as np
import pandas as pd
import math
import random
import copy

In [2]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [4]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [5]:
# define function to calculate the length of the tour 
def totalcost(tour, distance):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [6]:
# starting tour option

In [7]:
# define function to execute random tour
def random_tour(cities, distance):
    import numpy as np
    
    temp = [x for x in range(1,cities)]
    np.random.shuffle(temp)
    tour = [0] + temp + [0]
    length = totalcost(tour, distance)
    
    return (tour, length)

In [8]:
# define function to execute nearest neighbor        
def nearest_neighbor(cities, distance):
    import numpy as np

    position = 0
    tour = [0]
    length = 0
    for i in range(cities-1):
        nn = 0
        nd = np.nanmax(distance)+1
        for j in range(cities):  
            if (j not in tour) and (distance[position,j]<nd) and (i!=j):
                nd = distance[position,j]
                nn = j
        tour.append(nn)
        length = length + nd
        position = nn
    tour.append(0)
    length = length + distance[position,0]
    
    return (tour, length)

In [9]:
# define function to execute successive insertion
def succ_insert(cities, distance):
    import numpy as np

    tour = [0, 0] #start and end at node 0
    length = 0 #total length of current tour
    
    for i in range(1, cities) : #add nodes in order of 1 -> nodes-1
        length_change = np.zeros(len(tour) - 1) #list of change in length for each possible position to insert new node

        for position in range(len(tour) - 1): #iterate through possible positions in current tour
            #temp_tour = tour[: position+1] + [i] + tour[position+1 :]

            #calculate the change in length between current tour and temp tour
            pre_ins_node = tour[position]
            nex_ins_node = tour[position + 1]
            if pre_ins_node == nex_ins_node:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node]
            else:
                length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node] - distance[pre_ins_node][nex_ins_node]

        min_position = np.argmin(length_change) #get the position which create the shortest length change
        length += length_change[min_position]
        tour = tour[: min_position+1] + [i] + tour[min_position+1 :]
        
    return (tour, length)

In [10]:
#neighborhood definition options

In [11]:
#exchange
def exchange(tour, distance, tabu_list):
    delta_list = []
    for i in range(1, len(tour)-2):
        for j in range(i+1, len(tour)-1): 
            #if ([i,j] not in tabu_list) and ([j,i] not in tabu_list):  
            delta = distance[tour[i-1],tour[i]] + distance[tour[i],tour[i+1]] + distance[tour[j-1],tour[j]] + distance[tour[j],tour[j+1]] - distance[tour[i-1],tour[j]] - distance[tour[j],tour[i+1]] - distance[tour[j-1],tour[i]] - distance[tour[i],tour[j+1]]
            delta_list.append((delta, i, j))
            #improvement = positive delta

    #exchange node i and j
    ind = np.argmax([y[0] for y in delta_list])
    ind_i = delta_list[ind][1]
    ind_j = delta_list[ind][2]
    
    new_tour = copy.deepcopy(tour)
    new_tour[ind_i] = tour[ind_j]
    new_tour[ind_j] = tour[ind_i]
    
    return(new_tour, delta_list[ind][0], [ind_i, ind_j])


In [12]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [13]:
#2opt
def two_opt(tour, distance, tabu_list):
   
    best_tour = copy.deepcopy(tour)
    best_delta = 0
    improvement = True
    ind_i = 0
    ind_j = 0
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                #if ([i,j] not in tabu_list) and ([j,i] not in tabu_list):                 
                temp_tour = tour_swap(tour, i, j)
                delta = totalcost(tour[(i-1):(j+2)], distance) - totalcost(temp_tour[(i-1):(j+2)], distance) #improvement = positive delta

                if delta > best_delta:
                    best_tour = copy.deepcopy(temp_tour)
                    best_delta = delta
                    ind_i = i
                    ind_j = j
                    improvement = True
    
    return (best_tour, best_delta, [ind_i, ind_j])

In [14]:
#main function
def tabu_search (tour, length, distance, neighbor = 'two_opt', max_length = 20, aspiration='yes',
          TimeLimit = None, non_improvement = None, max_iteration = None):

    best = length
    besttour = tour
    best_iter = 0
    print(tour)
    print(best)
    tabu = []
    
    #time limit condition
    from time import process_time 
    start_time = process_time()
     
    #max iteration condition
    iteration = 0
    non_iter = 0
    
    terminated = False
    while not terminated: #termination condition     
    
        improvement = False
        iteration += 1     
    
        next_tour = []            
        if neighbor == 'exchange':
            (next_tour, best_delta, [best_i, best_j]) = exchange(tour, distance, tabu) #end = depot
        elif neighbor == 'two_opt':
            (next_tour, best_delta, [best_i, best_j]) = two_opt(tour, distance, tabu)
        else:
            print('Invalid neighborhood definition')
            break            

        if ([best_i, best_j] not in tabu) and ([best_j, best_i] not in tabu): 
                length = length - best_delta
                tour = next_tour
                tabu.append([best_i, best_j])
        else: #move included in tabu list
            if (aspiration == 'yes') and (best_delta > 0): 
                #aspiration criterion: waive tabu status of a move if the new solution is desirable
                if [best_i, best_j] in tabu: tabu.remove([best_i, best_j])
                if [best_j, best_i] in tabu: tabu.remove([best_j, best_i])

        # tabu list must be long enough to escape from local optima
        if len(tabu) > max_length: #maintain length of tabu list less or equal to max_length
            tabu.pop(0)

        if length < best:
            best = length
            besttour=tour
            best_iter = iteration
            improvement = True
            print(best)
                
        runtime = process_time() - start_time
        
        #check termination condition
        if (TimeLimit is not None): #time elapsed termination
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True

        elif (non_improvement is not None): #max iteration without new best solution
            if improvement: 
                non_iter = 0
            else: non_iter += 1
            if non_iter >= non_improvement:
                print('Reach max number of iteration without new best solution')
                terminated = True

        elif (max_iteration is not None):
            if iteration >= max_iteration:
                print('Reach max number of iteration')
                terminated = True
        else:
            if iteration >= 25: #default value
                print('Reach default max number of iteration (20)')
                terminated = True

    print("Terminate")
    print(besttour)
    print(best)
    
    return (best, iteration, runtime)

In [15]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

for i in range(cities):
    distance[i,i] = 10000

In [16]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [17]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

In [18]:
dataset = ((cities, distance), (cities1, distance1), (cities2, distance2))
index = ['Dataset 1', 'Dataset 2', 'Dataset 3']

**Notation**  
Dataset 1 = Example with 120 cities  
Dataset 2 = Example with 152 cities  
Dataset 3 = Example with 264 cities  

### Comparision between Starting solution  
Run 3 starting solution strategies: random, nearest neighbor, successive insertion with 3 datasets, the other parameters are default (max length of tabu list = 20, neighborhood definition = 'two_opt', aspiration criteria = 'yes'). Termination criteria is set as max number of iterations = 50.

In [19]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['random', 'nearest neighbor', 'successive insertion'])

#random starting tour
for i in range(3):
    (start_tour, start_length) = random_tour(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', max_length = 20, 
                          aspiration = 'yes', max_iteration = 25)
    value.iloc[i,0] = result

#nearest neighbor
for i in range(3):
    (start_tour, start_length) = nearest_neighbor(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', max_length = 20, 
                          aspiration = 'yes', max_iteration = 25)
    value.iloc[i,1] = result

#successive insertion
for i in range(3):
    (start_tour, start_length) = succ_insert(dataset[i][0], dataset[i][1])
    result = tabu_search (start_tour, start_length, dataset[i][1], neighbor = 'two_opt', max_length = 20, 
                          aspiration = 'yes', max_iteration = 25)
    value.iloc[i,2] = result

[0, 60, 9, 51, 35, 77, 79, 42, 29, 57, 82, 58, 59, 81, 73, 44, 106, 28, 2, 72, 45, 69, 34, 91, 84, 53, 85, 23, 17, 94, 24, 86, 103, 62, 71, 31, 1, 111, 49, 13, 3, 46, 54, 102, 41, 75, 104, 5, 67, 74, 89, 116, 109, 65, 22, 15, 48, 47, 114, 64, 32, 88, 87, 50, 30, 63, 95, 37, 110, 83, 55, 6, 21, 117, 52, 118, 112, 68, 19, 101, 99, 33, 18, 100, 90, 26, 105, 66, 10, 113, 8, 14, 93, 56, 115, 43, 107, 96, 61, 97, 92, 25, 108, 7, 78, 20, 11, 119, 36, 38, 76, 4, 16, 98, 39, 80, 12, 70, 40, 27, 0]
55063
53238
51580
50020
48462
46915
45436
44069
42668
41374
40116
38893
37619
36481
35411
34297
33314
32349
31418
30523
29655
28772
27991
27278
26510
25824
Reach max number of iteration
Terminate
[0, 60, 9, 5, 104, 75, 41, 102, 54, 46, 3, 13, 49, 45, 69, 34, 36, 38, 76, 4, 52, 118, 112, 68, 19, 48, 15, 22, 63, 95, 37, 110, 83, 55, 6, 21, 117, 16, 97, 31, 71, 62, 103, 35, 88, 87, 50, 8, 113, 10, 66, 105, 26, 79, 11, 20, 114, 47, 101, 100, 109, 33, 7, 108, 25, 92, 1, 111, 72, 2, 28, 29, 42, 90, 99, 78, 

75605.59000398744
75589.68908792139
Reach max number of iteration
Terminate
[0, 34, 35, 33, 47, 48, 49, 75, 73, 74, 72, 50, 46, 77, 90, 95, 114, 115, 94, 91, 76, 93, 92, 123, 124, 125, 151, 149, 150, 126, 127, 122, 121, 128, 129, 148, 147, 146, 130, 131, 120, 119, 132, 133, 145, 144, 143, 134, 135, 118, 117, 136, 137, 142, 141, 140, 138, 139, 116, 110, 111, 87, 102, 101, 100, 99, 112, 113, 98, 97, 96, 89, 78, 70, 71, 52, 51, 45, 69, 68, 53, 44, 79, 88, 80, 66, 67, 55, 54, 43, 65, 64, 56, 42, 81, 86, 103, 104, 105, 106, 109, 108, 107, 84, 83, 85, 82, 62, 63, 58, 57, 41, 61, 60, 59, 40, 39, 17, 16, 8, 7, 9, 18, 20, 19, 6, 10, 21, 22, 38, 5, 11, 23, 24, 25, 4, 12, 26, 27, 37, 3, 13, 28, 29, 30, 2, 14, 31, 32, 36, 1, 15, 0]
75589.68908792139
[0, 39, 40, 38, 37, 36, 60, 59, 120, 119, 118, 117, 116, 115, 62, 61, 35, 34, 33, 32, 31, 30, 64, 63, 114, 113, 112, 111, 110, 109, 66, 65, 29, 28, 27, 26, 25, 24, 68, 67, 108, 107, 106, 105, 104, 103, 70, 69, 23, 22, 21, 20, 19, 18, 72, 71, 102, 101, 

In [20]:
value1=value
print(value1)

                                        random  \
Dataset 1               (25824, 25, 34.890625)   
Dataset 2     (549856.4325270165, 25, 71.4375)   
Dataset 3  (741032.2829701921, 25, 367.953125)   

                              nearest neighbor  \
Dataset 1                (7263, 25, 33.578125)   
Dataset 2   (75589.68908792139, 25, 66.546875)   
Dataset 3  (53342.22519522212, 25, 299.046875)   

                         successive insertion  
Dataset 1             (7493.0, 25, 25.234375)  
Dataset 2  (79514.97420216419, 25, 67.640625)  
Dataset 3  (51535.90147988852, 25, 372.03125)  


#### Comment:
The performance of Nearest neighbor and Successive insertion is approximately the same. The Successive insertion works slightly worse than Nearest neighbor in dataset 1. In dataset 2, there is a mixed result, 1 test Successive insertion finds better solution and 1 test find worse one. In both tests, Successive insertion realizes tsolution quicker (fewer iterations needed) than Nearest neighbor.
However, the performance of Random starting tour is very different from the other two strategies. Random tour works worse in all datasets and in both tests, especially in large datasets. In general, the random starting tour is an unstable strategy. Therefore, it is safer to use heuristic starting tour than random one.

### **Comparision between Neighborhood definition**   
Run 3 Neighborhood definitions: 2-opt and exchange with 3 datasets. The starting solution is chosen as successive insertion, based on the previous comparision. The other parameters are default (initial = 0.01, cooling factor = 0.95, number of evaluation each iteration = 10000). The termination condition is set as TimeLimit = 60 seconds per dataset per definition.

In [22]:
start = []
for i in range(3):
    (start_tour, start_length) = succ_insert(dataset[i][0], dataset[i][1])
    start.append((start_tour, start_length))


In [24]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['two opt', 'exchange'])
neighbor_set = ['two_opt', 'exchange']


for j in range(2): #neighbor definition
    for i in range(3): #dataset
        result = tabu_search (start[i][0], start[i][1], dataset[i][1], neighbor = neighbor_set[j], max_length = 20, 
                          aspiration = 'yes', max_iteration = 25)
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
7732.0
7689.0
7654.0
7629.0
7604.0
7556.0
7548.0
7540.0
7530.0
7524.0
7513.0
7506.0
7505.0
7493.0
Reach max number of iteration
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 115, 69, 25, 70, 46, 54, 5, 83, 34, 9, 98, 61, 36, 66, 105, 103, 35, 111, 88, 47, 109, 100, 101, 2, 81, 52, 79, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 33, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 57, 7

56960.05492393449
56924.42580695983
56913.24118877456
Reach max number of iteration
Terminate
[0, 38, 33, 32, 31, 27, 25, 26, 21, 19, 20, 15, 13, 14, 9, 7, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 6, 80, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 10, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 200, 199, 259, 258, 202, 201, 194, 134, 135, 191, 190, 193, 192, 204, 203, 257, 256, 189, 206, 205, 255, 254, 260, 261, 262, 253, 252, 208, 207, 188, 187, 186, 136, 137, 185, 184, 183, 210, 209, 251, 250, 249, 248, 212, 211, 182, 181, 180, 138, 139, 179, 178, 177, 214, 213, 247, 246, 245, 244, 216, 215, 176, 175, 174, 140, 141, 171, 172, 173, 217, 218, 242, 243, 240, 241, 219, 220, 170, 142, 143, 165, 166, 169, 168, 167, 221, 222, 238, 239, 263, 236, 237, 223, 224, 162, 163, 164, 144, 145, 147, 146, 161, 160, 225, 159, 158, 157, 148, 154, 155, 156, 226, 227, 234, 235, 232, 231, 233, 228, 229, 152, 151, 230, 150, 153, 149, 130, 129, 99, 98, 97, 74, 73, 17, 16, 18, 72, 71, 102, 101

In [25]:
value2=value
print(value2)

                                      two opt  \
Dataset 1             (7493.0, 25, 25.203125)   
Dataset 2  (79514.97420216419, 25, 69.265625)   
Dataset 3   (51535.90147988852, 25, 356.8125)   

                                    exchange  
Dataset 1             (7693.0, 25, 0.453125)  
Dataset 2   (90174.97018727132, 25, 0.71875)  
Dataset 3  (56913.24118877456, 25, 2.328125)  


In [26]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['two opt', 'exchange'])
neighbor_set = ['two_opt', 'exchange']


for j in range(2): #neighbor definition
    for i in range(3): #dataset
        result = tabu_search (start[i][0], start[i][1], dataset[i][1], neighbor = neighbor_set[j], max_length = 20, 
                          aspiration = 'yes', max_iteration = 50)
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
7732.0
7689.0
7654.0
7629.0
7604.0
7556.0
7548.0
7540.0
7530.0
7524.0
7513.0
7506.0
7505.0
7493.0
Reach max number of iteration
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 115, 69, 25, 70, 46, 54, 5, 83, 34, 9, 98, 61, 36, 66, 105, 103, 35, 111, 88, 47, 109, 100, 101, 2, 81, 52, 79, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 33, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 57, 7

90174.97018727132
Reach max number of iteration
Terminate
[0, 34, 35, 15, 33, 36, 32, 14, 31, 30, 29, 13, 28, 37, 27, 12, 26, 25, 24, 11, 23, 10, 21, 22, 38, 47, 48, 49, 75, 73, 46, 50, 72, 74, 76, 92, 93, 94, 91, 77, 115, 114, 95, 90, 78, 71, 52, 45, 51, 70, 69, 68, 53, 44, 96, 97, 98, 89, 79, 113, 112, 99, 88, 80, 67, 55, 43, 54, 66, 65, 64, 56, 42, 100, 101, 102, 87, 81, 111, 110, 103, 86, 82, 63, 58, 41, 57, 62, 61, 60, 59, 104, 105, 106, 85, 83, 109, 123, 124, 125, 151, 149, 150, 126, 127, 122, 121, 128, 129, 148, 147, 146, 130, 131, 120, 119, 132, 133, 145, 144, 143, 134, 135, 118, 117, 136, 137, 142, 141, 140, 138, 139, 116, 108, 107, 84, 40, 19, 20, 9, 18, 39, 17, 16, 8, 7, 6, 5, 4, 3, 2, 1, 0]
90174.97018727132
[0, 38, 33, 32, 31, 27, 25, 26, 21, 19, 20, 15, 13, 14, 9, 7, 8, 3, 2, 1, 4, 5, 81, 83, 84, 87, 86, 85, 82, 6, 80, 79, 90, 89, 88, 93, 92, 91, 78, 77, 11, 10, 12, 76, 75, 96, 95, 94, 131, 132, 198, 133, 195, 196, 197, 200, 199, 259, 258, 202, 201, 194, 134, 135, 191, 19

In [27]:
value3=value
print(value3)

                                      two opt  \
Dataset 1             (7493.0, 50, 44.578125)   
Dataset 2  (77041.69242477352, 50, 126.28125)   
Dataset 3   (51484.404231826746, 50, 582.375)   

                                    exchange  
Dataset 1               (7693.0, 50, 0.9375)  
Dataset 2  (90174.97018727132, 50, 1.484375)  
Dataset 3   (56913.24118877456, 50, 4.78125)  


In [31]:
import pandas as pd
value = pd.DataFrame(index = index, columns = ['max_length = 5', 'max_length = 10', 'max_length = 25', 'max_length = 50'])
length_set = [5, 10, 25, 50]


for j in range(4): #neighbor definition
    for i in range(3): #dataset
        result = tabu_search (start[i][0], start[i][1], dataset[i][1], neighbor = 'two_opt', 
                              max_length = length_set[j], aspiration = 'yes', max_iteration = 50)
        value.iloc[i,j] = result

[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 69, 115, 33, 25, 70, 46, 88, 111, 35, 34, 9, 98, 105, 66, 36, 61, 103, 83, 5, 54, 47, 109, 100, 101, 2, 81, 79, 52, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 78, 57, 99, 32, 51, 107, 24, 18, 17, 84, 116, 30, 65, 80, 93, 21, 85, 74, 43, 86, 73, 13, 77, 44, 27, 91, 31, 58, 14, 29, 119, 28, 75, 0]
7779.0
7732.0
7689.0
7654.0
7629.0
7604.0
7556.0
7548.0
7540.0
7530.0
7524.0
7513.0
7506.0
7505.0
7493.0
Reach max number of iteration
Terminate
[0, 60, 15, 59, 23, 110, 95, 89, 53, 7, 115, 69, 25, 70, 46, 54, 5, 83, 34, 9, 98, 61, 36, 66, 105, 103, 35, 111, 88, 47, 109, 100, 101, 2, 81, 52, 79, 26, 4, 62, 113, 72, 56, 82, 38, 94, 96, 11, 76, 63, 87, 108, 20, 92, 1, 114, 10, 50, 12, 22, 8, 102, 118, 3, 33, 39, 104, 71, 37, 6, 55, 40, 41, 97, 117, 48, 16, 49, 45, 19, 106, 112, 68, 64, 42, 67, 90, 57, 7

82716.04046092476
82481.26497911337
82246.48949730198
82041.03942339894
81835.58934949589
81630.13927559284
81467.26029523836
81339.28740505739
81211.31451487642
81083.34162469544
81024.74672999432
80485.41959432916
80429.90314556714
80053.79192254176
80001.84424700092
79949.89657146008
79899.58231053529
79753.04438148408
79702.73012055931
79556.1921915081
79514.97420216419
79186.46560858445
78893.18871065782
78855.31070195386
78817.4326932499
78779.55468454593
78741.67667584197
78726.88707854187
77872.4647989292
77866.72445804498
77595.54067421064
77589.80033332642
77318.61654949209
77312.87620860786
77041.69242477353
77041.69242477352
Reach max number of iteration
Terminate
[0, 15, 1, 14, 2, 13, 3, 12, 4, 11, 5, 10, 6, 19, 20, 18, 9, 7, 8, 16, 17, 39, 40, 59, 60, 104, 105, 106, 85, 83, 84, 107, 108, 109, 116, 139, 138, 140, 141, 142, 137, 136, 117, 118, 135, 134, 143, 144, 145, 133, 132, 119, 120, 131, 130, 146, 147, 148, 129, 128, 121, 122, 127, 126, 150, 149, 151, 125, 124, 123, 92

56938.766812014095
56627.94203415031
56007.567306095254
55478.341710432265
55162.113944415425
54729.658412381745
54335.400905861796
54019.173139844956
53709.271188485676
53458.13376493658
53258.13376493658
53088.41132870978
52964.8045309598
52841.197733209825
52724.04044568444
52441.197733209825
52341.197733209825
52280.321502949526
52219.766375403124
52165.29584600328
52129.66672902862
51965.29584600328
51682.45313352867
51558.18906481674
51535.90147988852
51513.6138949603
51506.23486322715
51498.85583149401
51491.476799760865
51484.40423182675
51484.404231826746
Reach max number of iteration
Terminate
[0, 40, 41, 42, 43, 44, 45, 48, 47, 46, 51, 50, 49, 55, 52, 53, 54, 126, 125, 127, 124, 123, 122, 121, 57, 56, 58, 39, 37, 36, 60, 59, 120, 119, 118, 128, 149, 153, 152, 151, 150, 230, 229, 228, 233, 231, 232, 235, 234, 227, 226, 156, 155, 154, 148, 157, 158, 159, 225, 160, 161, 147, 146, 145, 144, 164, 163, 162, 224, 223, 237, 236, 263, 239, 238, 222, 221, 167, 166, 165, 143, 142, 170,

In [32]:
value4=value
print(value4)

                                max_length = 5  \
Dataset 1                  (7493.0, 50, 44.75)   
Dataset 2  (77041.69242477352, 50, 131.828125)   
Dataset 3   (51484.404231826746, 50, 604.3125)   

                              max_length = 10  \
Dataset 1             (7493.0, 50, 42.390625)   
Dataset 2  (77041.69242477352, 50, 127.15625)   
Dataset 3  (51484.404231826746, 50, 595.8125)   

                              max_length = 25  \
Dataset 1             (7493.0, 50, 43.671875)   
Dataset 2  (77041.69242477352, 50, 130.71875)   
Dataset 3   (51484.404231826746, 50, 608.625)   

                                max_length = 50  
Dataset 1                (7493.0, 50, 45.71875)  
Dataset 2    (77041.69242477352, 50, 131.28125)  
Dataset 3  (51484.404231826746, 50, 623.453125)  


In [21]:
# def tabu_search (tour, length, distance, neighbor = 'random', max_length = 20, aspiration='yes',
#           TimeLimit = None, non_improvement = None, max_iteration = None):
